In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ultralytics opencv-python-headless

In [ ]:
# Install ffmpeg for better video codec support
!apt-get update
!apt-get install -y ffmpeg

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 2s (2,169 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (

In [ ]:
import os
import sys

# Your code directory
CODE_DIR = "/content/drive/MyDrive/orbro/code"

# Change to code directory
os.chdir(CODE_DIR)
print(f"✅ Changed to directory: {CODE_DIR}")

# Add to Python path so imports work
if CODE_DIR not in sys.path:
    sys.path.insert(0, CODE_DIR)

# List files in directory
print("\nFiles in code directory:")
for f in sorted(os.listdir(CODE_DIR)):
    if f.endswith('.py'):
        print(f"  ✓ {f}")

In [ ]:
required_files = ['config.py', 'utils.py', 'tracker.py', 'fall_analyzer.py', 'main.py', 'test_video.py']
missing = []

print("\nVerifying files...")
for f in required_files:
    if os.path.exists(os.path.join(CODE_DIR, f)):
        print(f"  ✅ {f}")
    else:
        print(f"  ❌ {f} - MISSING!")
        missing.append(f)

if missing:
    print(f"\n⚠️  ERROR: Missing files: {', '.join(missing)}")
    print("Please upload these files to /content/drive/MyDrive/orbro/code")
else:
    print("\n✅ All required files present!")

In [ ]:
# # Edit config.py with your paths, or run this to update them:

# config_content = '''"""
# config.py - Configuration settings for fall detection system
# """

# # ================= PATHS =================
# MODEL_PATH_DETECTION = "yolov8s.pt"
# MODEL_PATH_POSE = "yolov8s-pose.pt"
# INPUT_DIR = "/content/drive/MyDrive/orbro/Fall_vid"
# OUTPUT_DIR = "/content/drive/MyDrive/orbro/Fall_vid_output"

# # ================= DETECTION THRESHOLDS =================
# CONF_THRESH = 0.4
# POSE_CONF_THRESH = 0.5
# MIN_AREA_RATIO = 0.002
# MAX_MISSING = 20
# MIN_KEYPOINTS_VISIBLE = 5

# # ================= MOTION THRESHOLDS =================
# STATIC_MOVE_THRESH = 3
# MIN_LIVING_FRAMES = 20
# MANNEQUIN_MOTION_THRESH = 0.5
# MANNEQUIN_CHECK_FRAMES = 30
# MANNEQUIN_JITTER_THRESH = 1.0

# # ================= FALL DETECTION THRESHOLDS =================
# STANDING_CONFIDENCE_THRESH = 0.85
# FALLING_CONFIDENCE_THRESH = 0.70
# FALLEN_CONFIDENCE_THRESH = 0.60
# MIN_FALL_CONFIDENCE = 0.60

# HEIGHT_DROP_RATIO = 0.30
# ASPECT_RATIO_CHANGE = 1.5
# VEL_Y_THRESH = 20
# ACC_Y_THRESH = 15
# BODY_ANGLE_THRESH = 50
# POST_FALL_STATIC_FRAMES = 30
# FALL_CONFIRM_FRAMES = 3

# # ================= OCCLUSION & BODY PART TRACKING =================
# OCCLUSION_AREA_RATIO = 0.5
# OCCLUSION_KEYPOINT_RATIO = 0.4
# SUDDEN_DISAPPEAR_FRAMES = 5
# REAPPEAR_GROUND_THRESH = 0.3
# BODY_PART_MATCH_THRESH = 50
# MIN_BODY_PART_MOVEMENT = 2.0

# # ================= VISUALIZATION =================
# COLOR_STANDING = (0, 255, 0)
# COLOR_FALLING = (0, 165, 255)
# COLOR_FALLEN = (0, 0, 255)
# COLOR_KEYPOINT = (255, 255, 0)
# COLOR_BODY_PART = (255, 0, 255)
# COLOR_MANNEQUIN = (128, 128, 128)
# '''

# with open('config.py', 'w') as f:
#     f.write(config_content)

# print("✅ config.py updated with default paths")
# print("\n⚠️  Make sure to edit INPUT_DIR and OUTPUT_DIR to match your Google Drive structure!")


In [ ]:
from config import INPUT_DIR, OUTPUT_DIR

print("\nConfiguration:")
print(f"  Input directory:  {INPUT_DIR}")
print(f"  Output directory: {OUTPUT_DIR}")

# Check if input directory exists
if os.path.exists(INPUT_DIR):
    video_files = [f for f in os.listdir(INPUT_DIR) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
    print(f"\n✅ Input directory exists")
    print(f"   Found {len(video_files)} video file(s)")
    for vf in video_files:
        print(f"     - {vf}")
else:
    print(f"\n❌ Input directory not found: {INPUT_DIR}")
    print("   Please update INPUT_DIR in config.py")

# Check if output directory exists (create if not)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"\n✅ Created output directory: {OUTPUT_DIR}")
else:
    print(f"\n✅ Output directory exists: {OUTPUT_DIR}")

In [ ]:
from ultralytics import YOLO

print("Downloading YOLO models (this may take a minute)...")
try:
    detection_model = YOLO('yolov8s.pt')
    pose_model = YOLO('yolov8s-pose.pt')
    print("✅ Models downloaded and ready!")
except Exception as e:
    print(f"❌ Error downloading models: {e}")

In [ ]:
print("\n" + "="*60)
print("TESTING VIDEO CODEC COMPATIBILITY")
print("="*60)
print("This will test which video codecs work on this system...")
print()

!python test_video.py

print("\n⚠️  Check the output above to ensure at least one codec works!")

In [ ]:
print("Testing imports...")

from config import *
from utils import *
from tracker import new_track, new_body_part_track
from fall_analyzer import analyze_fall_indicators

print("✅ All imports OK")


In [ ]:
print("\n" + "="*60)
print("STARTING FALL DETECTION PROCESSING")
print("="*60)
print("\nThis will process all videos in the input directory.")
print("\n")
!python main.py

In [ ]:
import os
from config import OUTPUT_DIR

print("\n" + "="*60)
print("CHECKING OUTPUT FILES")
print("="*60)

if os.path.exists(OUTPUT_DIR):
    output_files = [f for f in os.listdir(OUTPUT_DIR) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

    print(f"\nOutput directory: {OUTPUT_DIR}")
    print(f"Files created: {len(output_files)}\n")

    for f in sorted(output_files):
        filepath = os.path.join(OUTPUT_DIR, f)
        size = os.path.getsize(filepath)
        size_mb = size / (1024 * 1024)
        print(f"  ✓ {f} ({size_mb:.2f} MB)")

    if not output_files:
        print("⚠️  No output files found!")
        print("Check the error messages above for issues.")
else:
    print(f"❌ Output directory not found: {OUTPUT_DIR}")

In [ ]:
from IPython.display import Video
from config import OUTPUT_DIR
import os

output_files = [f for f in os.listdir(OUTPUT_DIR) if f.lower().endswith(('.mp4', '.avi'))]
if output_files:
    sample_video = os.path.join(OUTPUT_DIR, output_files[0])
    print(f"Playing: {output_files[0]}")
    display(Video(sample_video, width=800))
else:
    print("No videos to display")

In [ ]:
# import os
# from config import OUTPUT_DIR

# if os.path.exists(OUTPUT_DIR):
#     files = os.listdir(OUTPUT_DIR)
#     if files:
#         print(f"Deleting {len(files)} files from {OUTPUT_DIR}...")
#         for f in files:
#             file_path = os.path.join(OUTPUT_DIR, f)
#             try:
#                 os.remove(file_path)
#                 print(f"  - Removed {f}")
#             except Exception as e:
#                 print(f"  - Error removing {f}: {e}")
#         print("✅ All files deleted.")
#     else:
#         print(f"⚠️  {OUTPUT_DIR} is already empty.")
# else:
#     print(f"❌ Output directory not found: {OUTPUT_DIR}")


